In [ ]:
# Libraries

! pip install --user pycm

import os, cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical

import os, cv2
from tqdm import tqdm

from pycm import *

In [ ]:
X=[]
Z=[]

IMG_SIZE=256

ACCENT_ENGLISH_DIR='../input/accent-data/english/'
ACCENT_FRENCH_DIR='../input/accent-data/french/'
ACCENT_GERMAN_DIR='../input/accent-data/german/'
ACCENT_HINDI_DIR='../input/accent-data/hindi/'
ACCENT_ARABIC_DIR='../input/accent-data/arabic/'

In [ ]:
def target(img, nationality):
    return nationality

In [ ]:
def data_creation(nationality,DIR):
    for img in tqdm(os.listdir(DIR)):
        name = img
        label = target(img,nationality)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append([str(label),name])

In [ ]:
# Creation of the data corresponding to the 5 accents

data_creation('english',ACCENT_ENGLISH_DIR)
data_creation('french',ACCENT_FRENCH_DIR)
data_creation('german',ACCENT_GERMAN_DIR)
data_creation('hindi',ACCENT_HINDI_DIR)
data_creation('arabic',ACCENT_ARABIC_DIR)

In [ ]:
def equal_list(l1, l2):
    for i in range(len(l2)):
        if l1[i] != l2[i]:
            return False
    return True

In [ ]:
le = LabelEncoder()
L = []
# We transform the data in a way understandable by a Deep-Learning model

le = LabelEncoder()
L = []
for i in range(len(Z)):
    L.append(Z[i][0])
Y = le.fit_transform(L)
Y = to_categorical(Y,5)
l = []
for i in range(len(Y)):
    l.append([Y[i], Z[i][1]])
Y = l
X = np.array(X)
X = X/255

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)

# **SVM**

In [ ]:
# We build y_train and y_test (with just the labels)

train_list = []
for l in y_train:
    train_list.append(l[0])

test_list = []
for l in y_test:
    test_list.append(l[0])

y_train = train_list
y_test = test_list

In [ ]:
for i in range(len(y_train)):
    y_train[i] = list(y_train[i])
y_train = np.asarray(y_train)

for i in range(len(y_test)):
    y_test[i] = list(y_test[i])
y_test = np.asarray(y_test)

list_train = []

for i in range(len(y_train)):
    if equal_list([1., 0., 0., 0., 0.], y_train[i]):
        list_train.append(1)
    elif equal_list([0., 1., 0., 0., 0.], y_train[i]):
        list_train.append(2)
    elif equal_list([0., 0., 1., 0., 0.], y_train[i]):
        list_train.append(3)
    elif equal_list([0., 0., 0., 1., 0.], y_train[i]):
        list_train.append(4)
    elif equal_list([0., 0., 0., 0., 1.], y_train[i]):
        list_train.append(5)
y_train = list_train

list_test = []

for i in range(len(y_test)):
    if equal_list([1., 0., 0., 0., 0.], y_test[i]):
        list_test.append(1)
    elif equal_list([0., 1., 0., 0., 0.], y_test[i]):
        list_test.append(2)
    elif equal_list([0., 0., 1., 0., 0.], y_test[i]):
        list_test.append(3)
    elif equal_list([0., 0., 0., 1., 0.], y_test[i]):
        list_test.append(4)
    elif equal_list([0., 0., 0., 0., 1.], y_test[i]):
        list_test.append(5)
y_test = list_test

In [ ]:
x_train = x_train.reshape(len(x_train), -1)

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_train, y_train)

In [ ]:
x_test = x_test.reshape(len(x_test), -1)
y_pred = clf.predict(x_test)

In [ ]:
cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred)
cm.relabel(mapping={1:"arabic", 2:"english", 3:"french", 4:"german", 5:"hindi"})
print(cm)